In [2]:
from analyst import Analyst
from config import ModelConfig, TrainerConfig
from dataset import load_dataset_manager

In [22]:
trainer_config = TrainerConfig(
    dataset_name="toydata", epochs=1, ignore_saved_model=True, load_model=False, batch_size=64
)
model_config = ModelConfig(d_model=128, lr=0.005)

In [23]:
trainer_config

TrainerConfig(model_name='attentive2', dataset_name='toydata', epochs=1, batch_size=64, verbose=False, ignore_saved_model=True, load_model=False, save_model=True, load_dataset=True, save_dataset=True, cache_dir='cache/', dataset_dir='cache/dataset/')

In [24]:
model_config

ModelConfig(d_model=128, max_embedding_norm=None, window_size=8, negative_sample_size=5, lr=0.005, use_learnable_embedding=True, dropout=0.1, add_seq_embedding=False, add_positional_encoding=False)

In [10]:
dataset_manager = load_dataset_manager(
    dataset_name=trainer_config.dataset_name,
    dataset_dir=trainer_config.dataset_dir,
    load_dataset=trainer_config.load_dataset,
    save_dataset=trainer_config.save_dataset,
)

load cached dataset_manager from: cache/dataset/toydata.pickle


In [25]:
analyst = Analyst(
    dataset_manager=dataset_manager,
    trainer_config=trainer_config,
    model_config=model_config,
)

In [26]:
analyst.fit(show_fig=False)

train start


  3%|▎         | 213/7313 [00:03<01:45, 67.34it/s]


KeyboardInterrupt: 